# Retrieval Augmented Generation

RAG combines search (retrieval) with AI text generation. First retrieve relevant documents, then use them to generate informed responses.

![images/llm_2_rag_basic.png](images/llm_2_rag_basic.png)

## Connect to Weaviate

Connect to a Weaviate instance with third party credentials as needed.

In [ ]:
# Refresh credentials & load the Weaviate IP
from helpers import update_creds

AWS_ACCESS_KEY, AWS_SECRET_KEY, AWS_SESSION_TOKEN = update_creds()

%store -r WEAVIATE_IP

In [1]:
import weaviate

client = weaviate.connect_to_local(
    WEAVIATE_IP,
    headers = {
        "X-AWS-Access-Key": AWS_ACCESS_KEY,
        "X-AWS-Secret-Key": AWS_SECRET_KEY,
        "X-AWS-Session-Token": AWS_SESSION_TOKEN,
    }        
)

client.is_ready()

True

### Start with (R) - Retrieval

First step: retrieve relevant documents using search.

In [2]:
articles = client.collections.use("FinancialArticles")

response = articles.query.near_text(
    query="cloud revenue",
    limit=5,
    target_vector="title",
)

for item in response.objects:
    print(item.properties["article_title"])

Accenture forecasts fourth-quarter revenue below estimates on forex hit
Cirrus Logic (CRUS) Q2 Earnings & Revenues Top, Shares Up
Oracle, FedEx, UPS, Adobe and General Mills are part of Zacks Earnings Preview
BYD posts 350% jump in Q3 net profit on sales surge
Applied Materials (AMAT) Surpasses Q1 Earnings and Revenue Estimates


### Add (AG) - augmented generation - to make full RAG

Second step: use retrieved documents to generate informed responses.

#### Single Prompt

> Generate a response per **retrieved** object.

In [3]:
# Let's add some colour to our lives :)
BLUE   = "\033[94m"
PURPLE = "\033[95m"
RESET  = "\033[0"

In [4]:
from weaviate.classes.generate import GenerativeConfig

articles = client.collections.use("FinancialArticles")

response = articles.generate.near_text(
    query="cloud revenue",
    limit=5,
    target_vector="title",
    generative_provider=GenerativeConfig.openai(
        model="gpt-4o-mini",
    ),
    single_prompt="Summarize this article to a sentence or two: {article}"
)

for item in response.objects:
    print(f"{BLUE}=== Source ===")
    print(item.properties["article_title"])

    print(f"{PURPLE}=== Generated Response ===")
    print(item.generative.text)
    print("\n")

=== Source ===
Accenture forecasts fourth-quarter revenue below estimates on forex hit
=== Generated Response ===
Accenture Plc forecasted fourth-quarter revenue between $15.0 billion and $15.5 billion, below analyst expectations of $15.70 billion, due to the negative impacts of high inflation and a stronger dollar on its international earnings. As a result, shares of the company fell 2.8% in pre-market trading.


=== Source ===
Cirrus Logic (CRUS) Q2 Earnings & Revenues Top, Shares Up
=== Generated Response ===
Cirrus Logic, Inc. reported better-than-expected second-quarter results, with non-GAAP earnings per share of $1.36 exceeding estimates, despite a slight year-over-year revenue decline to $425.5 million. The company provided an optimistic revenue outlook for the third quarter, anticipating revenues between $510 million and $550 million, driven by solid customer relationships and ongoing investments in the audio segment, although potential risks include reliance on Apple Inc. and

#### Grouped Task

Generate one response using all retrieved documents together.

In [5]:
response = articles.generate.near_text(
    query="cloud revenue",
    limit=5,
    target_vector="title",
    generative_provider=GenerativeConfig.openai(model="gpt-4o-mini"),
    grouped_task="Explain top 3 or so common trends or overall learnings in these articles. Please only use the provided content."
)

print(f"{PURPLE}=== Generated Response ===")
print(response.generative.text)

print(f"{BLUE}=== Source ===")
for item in response.objects:
    print(item.properties["article_title"])

=== Generated Response ===
Here are the top three common trends or overall learnings from the provided articles:

1. **Impact of Economic Factors on Revenue Outlook**: Several articles highlight how external economic conditions, such as inflation and currency fluctuations, significantly affect companies' revenue forecasts. For instance, Accenture faced a revenue forecast below expectations primarily due to high inflation and a stronger dollar impacting its international earnings. Similarly, economic growth dynamics were noted to influence other sectors, affecting revenues and earnings estimates.

2. **Strong Earnings Surprises Amid Challenging Conditions**: Multiple companies reported better-than-expected earnings, signifying resilience despite broader economic challenges. Cirrus Logic and Applied Materials, for example, both surpassed earnings estimates. This trend underscores the ability of certain firms to perform well through strategic positioning, innovation, or strong customer re

![images/llm_3_rag_weaviate.png](images/llm_3_rag_weaviate.png)

#### Specify which properties to use for grouped task

Control which document fields the AI uses for generation.

In [6]:
response = articles.generate.near_text(
    query="artificial intelligence",
    limit=50,
    target_vector="title",
    grouped_task="What are the top 3 most common types of articles, based on these titles? Please only use the provided content.",
    grouped_properties=["article_title"],

    generative_provider=GenerativeConfig.openai(
        model="gpt-4o-mini",
    ),
)

print(f"{PURPLE}=== Generated Response ===")
print(response.generative.text)

print(f"{BLUE}=== Source ===")
for item in response.objects:
    print(item.properties["article_title"])

=== Generated Response ===
Based on the provided titles, the top 3 most common types of articles are:

1. **Earnings Reports and Forecasts**: Articles that discuss company earnings and financial performance, such as "Analog Devices (ADI) Q4 Earnings to Ride on End Market Growth" and "Applied Materials (AMAT) Surpasses Q1 Earnings and Revenue Estimates."

2. **Stock Performance Analysis**: Articles that analyze specific stocks or companies, such as "Why Shares of Advanced Micro Devices Surged Today" and "Why Shopify Stock Was Popping Today."

3. **Market Trends and Sector Updates**: Articles that provide insights into market trends or updates on specific sectors, such as "Technology Sector Update for 07/14/2023: ASML, WDC, META, OPRA" and "US STOCKS-Wall Street falls for third day on inflation jitters, crypto plunge."
=== Source ===
Nvidia Dominates AI Silicon, But That Doesn't Justify A $1 Trillion Valuation
How to Invest in Big Tech’s AI Chatbot Craze
How Microsoft and Google Are Turn

## Set default Generative model

Configure a default model to avoid specifying it each time.

In [7]:
from weaviate.classes.config import Reconfigure

articles.config.update(
    generative_config=Reconfigure.Generative.openai(
        model="gpt-4o-mini"  # Update the generative model
    )
)

Try generative query without specifying the model (uses default).

In [8]:
response = articles.generate.near_text(
    query="cryptocurrency regulation",
    limit=5,
    target_vector="title",
    grouped_task="Briefly, what topics are covered in these articles? Please only use the provided content.",
)

print(f"{PURPLE}=== Generated Response ===")
print(response.generative.text)

print(f"{BLUE}=== Source ===")
for item in response.objects:
    print(item.properties["article_title"])

=== Generated Response ===
The articles cover the following topics:

1. **US Stocks and Cryptocurrency Impact**: The first article discusses how Wall Street indexes fell due to a significant drop in cryptocurrencies following China's ban on crypto services, as well as concerns over inflation affecting equity markets.

2. **Technology Sector Update**: The second article provides an update on major tech companies, including issues faced by ASML regarding chipmaking equipment sales in China, merger talks between Western Digital and Kioxia, and declining user engagement with Meta's Threads app.

3. **Economic Developments Around Independence Day**: The third article reviews economic indicators leading up to the Fourth of July celebrations, noting a decrease in inflation, increased consumer spending, and potential economic resilience, along with sector opportunities tied to the holiday.

4. **TC Energy's Expansion Project**: The fourth article discusses TC Energy Corporation's approval for 

## Close the client

Always close your connection when finished.

In [9]:
client.close()